In [ ]:
import pypsa
import pandas as pd
import matplotlib.pyplot as plt

plt.style.use("bmh")

In [ ]:
year = 2030
url = f"https://raw.githubusercontent.com/PyPSA/technology-data/master/outputs/costs_{year}.csv"
costs = pd.read_csv(url, index_col=[0, 1])

In [ ]:
costs.loc[costs.unit.str.contains("/kW"), "value"] *= 1e3
costs.unit = costs.unit.str.replace("/kW", "/MW")

defaults = {
    "FOM": 0,
    "VOM": 0,
    "efficiency": 1,
    "fuel": 0,
    "investment": 0,
    "lifetime": 25,
    "CO2 intensity": 0,
    "discount rate": 0.07,
}
costs = costs.value.unstack().fillna(defaults)

costs.at["OCGT", "fuel"] = costs.at["gas", "fuel"]
costs.at["CCGT", "fuel"] = costs.at["gas", "fuel"]
costs.at["OCGT", "CO2 intensity"] = costs.at["gas", "CO2 intensity"]
costs.at["CCGT", "CO2 intensity"] = costs.at["gas", "CO2 intensity"]

In [ ]:
def annuity(r, n):
    return r / (1.0 - 1.0 / (1.0 + r) ** n)

In [ ]:
costs["marginal_cost"] = costs["VOM"] + costs["fuel"] / costs["efficiency"]

In [ ]:
annuity = costs.apply(lambda x: annuity(x["discount rate"], x["lifetime"]), axis=1)

In [ ]:
costs["capital_cost"] = (annuity + costs["FOM"] / 100) * costs["investment"]

### time series

In [ ]:
url = (
    "https://tubcloud.tu-berlin.de/s/pKttFadrbTKSJKF/download/time-series-lecture-2.csv"
)
ts = pd.read_csv(url, index_col=0, parse_dates=True)

In [ ]:
ts.load *= 1e3

### network

In [ ]:
n = pypsa.Network()

In [ ]:
n.add("Bus", "electricity")

In [ ]:
n.set_snapshots(ts.index)

In [ ]:
n.snapshots

In [ ]:
n.snapshot_weightings.head(3)

### Adding Components

In [ ]:
carriers = [
    "OCGT",
    "CCGT",
]

n.add(
    "Carrier",
    carriers,
    color=["dodgerblue", "aquamarine"],
)

In [ ]:
n.add(
    "Load",
    "demand",
    bus="electricity",
    p_set=ts.load,
)

In [ ]:
n.loads_t.p_set.plot()

In [ ]:
n.add(
    "Generator",
    "OCGT",
    bus="electricity",
    carrier="OCGT",
    capital_cost=costs.at["OCGT", "capital_cost"],
    marginal_cost=costs.at["OCGT", "marginal_cost"],
    efficiency=costs.at["OCGT", "efficiency"],
    p_nom_extendable=True,
    # p_nom=20000,
)

n.add(
    "Generator",
    "CCGT",
    bus="electricity",
    carrier="CCGT",
    capital_cost=costs.at["CCGT", "capital_cost"],
    marginal_cost=costs.at["CCGT", "marginal_cost"],
    efficiency=costs.at["CCGT", "efficiency"],
    p_nom_extendable=True,
    # p_nom=60000,
)

### Run

In [ ]:
n.optimize(solver_name="gurobi")

### Let's take a look

In [ ]:
n.objective + n.objective_constant

In [ ]:
(n.statistics.capex() + n.statistics.opex()).sum()

In [ ]:
n.generators_t.p.sum(axis=1) @ n.buses_t.marginal_price

In [ ]:
n.buses_t.marginal_price[:].plot()

In [ ]:
n.statistics.revenue().sum()